In [1]:
import numpy as np
import os
import os.path
import datagenerate
from importlib import reload

#sigmoid function

def sigmoid(z):
    """
    compute the sigmoid of z
    
    input:
    z: numpy array
    
    output:
    s: numpy array
    
    """
    
    s = 1/(1+np.exp(-z))
    return s


def initialize_parameters(layer_dim):
    """
    initialize the parameters and return it
    
    """
    
    #np.random.seed(1)
    nn = len(layer_dim)
    parameters = {}
    
    for i in range(1, nn):
        parameters["W" + str(i)] = np.random.randn(layer_dim[i], layer_dim[i-1])*0.01
        parameters["b" + str(i)] = np.zeros((layer_dim[i], 1))
                            
    return parameters


def forward_propagation(X, parameters):
    """
    Compute the forward propagation and store intermediate cache
    
    """
    
    layer = int(len(parameters)/2)
    cache = {}
    cache["A0"] = X
    
    for i in range(1, layer):
        W = parameters["W" + str(i)]
        b = parameters["b" + str(i)]
        A = cache["A" + str(i-1)]
        Zi = np.dot(W, A) + b
        Ai = np.tanh(Zi)
        cache["Z" + str(i)] = Zi
        cache["A" + str(i)] = Ai
    
    W = parameters["W" + str(layer)]
    b = parameters["b" + str(layer)]
    A = cache["A"+str(layer-1)]
    Zi = np.dot(W, A) + b
    Ai = sigmoid(Zi)
    cache["Z" + str(layer)] = Zi
    cache["A" + str(layer)] = Ai
    
    return Ai, cache
    
    
def my_cost(A, Y):
    """
    Cost function
    """
    
    m = A.shape[1]
    
    assert(A.shape[1] == Y.shape[1])
    
    cost = -1/m*np.sum(np.log(A)*Y + (1-Y)*np.log(1-A))
    
    return cost


def back_propagation(parameters, cache, X, Y):
    """
    Caculate the grads of the Neural Network
    """
    
    m = X.shape[1]
    grad = {}
    #cache["A0"]
    dim = int(len(parameters)/2)
    A = cache["A" + str(dim)]
    dZ = A - Y
    dW = 1/m*np.dot(dZ,(cache["A"+str(dim-1)]).T)
    db = 1/m*np.sum(dZ, axis=1, keepdims=True)
    grad["dW"+str(dim)] = dW
    grad["db"+str(dim)] = db
    
    dZ_next = dZ
    for i in range(dim-1, 0, -1):
        A_cur = cache["A"+str(i)]
        A_pre = cache["A"+str(i-1)]
        W_next = parameters["W"+str(i+1)]
        
        dZ = np.dot(W_next.T, dZ_next)*(1-np.power(A_cur,2))
        dW = 1/m*np.dot(dZ, A_pre.T)
        db = 1/m*np.sum(dZ, axis=1, keepdims=True)
        
        dZ_next = dZ
        grad["dW"+str(i)] = dW
        grad["db"+str(i)] = db
    
    return grad


def update_parameters(parameters, grad, learning_rate=1.2):
    """
    update parameters with inputs:
    parameters: Neural Network parameters
    grad: derivative of the parameter
    learning_rate: hyperparameter
    
    output:
    Neural Network Parameters
    """
    
    dim = int(len(parameters)/2)
    for i in range(1, dim+1):
        parameters["W" + str(i)] -= learning_rate*grad["dW"+str(i)]
        parameters["b" + str(i)] -= learning_rate*grad["db"+str(i)]
        
    return parameters

    
def nn_model(X, Y, dim, num_iteration = 1000, learning_rate=1.2):
    """
    building nueral network with previous function
    inputs:
    X,Y: training data of inputs and outputs
    dim: Neural Network structure
    num_iteration: hyperparameter
    learning_rate: hyperparameter
    
    outputs:
    parameter
    """
    
    parameters = initialize_parameters(dim)
    
    for i in range(num_iteration):
        Ai, cache = forward_propagation(X,parameters)
        grad = back_propagation(parameters, cache, X, Y)
        parameters = update_parameters(parameters, grad, learning_rate)
    
    return parameters


def predict(X, parameters):
    """
    use the trained parameter with test inputs to predict the output.
    """
    
    Ai, cache = forward_propagation(X, parameters)
    Ai = Ai > 0.5
    
    return Ai



In [2]:
## Construct the input data
## directory data1 contain the true inputs
## directory data2 contain the false inputs

X2 = datagenerate.generateNNInputs("data2")
X1 = datagenerate.generateNNInputs("data1")

Y1 = np.ones((1, X1.shape[1]))
Y2 = np.zeros((1, X2.shape[1]))

X = np.column_stack((X1, X2))
Y = np.column_stack((Y1, Y2))

In [3]:
## construct the four layer Neural Network 
dim = [2700, 100, 50, 1]

## Train the Neural Network model
parameters = nn_model(X, Y, dim, num_iteration=4000, learning_rate=0.1)

In [4]:
## test the parameter
## directory test1 contain the true inputs

X_test = datagenerate.generateNNInputs("test1")
Y_test = predict(X_test, parameters)
Y_test

array([[ 0.99996025,  0.99995704,  0.99995792]])

In [5]:
## directory test1 contain the false inputs

X_test = datagenerate.generateNNInputs("test2")
Y_test = predict(X_test, parameters)
Y_test

array([[  6.13306974e-04,   6.97000670e-05,   8.47383881e-05]])